# 🎙️ AI 팟캐스트 생성 시스템

특정 카테고리를 입력 받아 그 카테고리에 대한 팟캐스트를 생성하는 모델을 구현합니다.

**워크플로우:**
1. 데이터 수집
2. 보고서 형태로 정리 및 요약
3. 팟캐스트 대본 생성
4. STT 모델 활용 팟캐스트 생성
5. 음성파일 전처리
6. 팟캐스트 리턴


## 📦 필요한 라이브러리 임포트


In [2]:
import json
import os
from datetime import datetime
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
import warnings
warnings.filterwarnings('ignore')
from langchain_google_genai import ChatGoogleGenerativeAI

# TTS 관련 라이브러리
from google.cloud import texttospeech
import io
import wave
from pydub import AudioSegment
from pydub.effects import normalize
import re

# 환경 변수 설정 (실제 사용시 .env 파일에서 로드)
import os
from dotenv import load_dotenv
load_dotenv()
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

# Google Cloud 서비스 계정 키 파일 경로 설정
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'env/gen-lang-client-0427413547-fe02748a16b5.json'

# Gemini 모델 초기화
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",  # 빠르고 비용 효율적인 모델
    temperature=0.7,
    google_api_key=GOOGLE_API_KEY
)

# Google TTS 클라이언트 초기화
tts_client = texttospeech.TextToSpeechClient()

## 📊 JSON 데이터 읽기 및 확인


In [3]:
# JSON 파일 읽기
with open('data/input/test.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

# 데이터 구조 확인
print(f"카테고리: {data['category']}")
for idx, article in enumerate(data['articles']):
    print(f"기사 {idx+1}:")
    print(f"제목: {article['title']}")
    print(f"날짜: {article['date']}")
    print(f"내용: {article['text'][:100]}...")
    print("\n")


카테고리: IT/기술
기사 1:
제목: 구글, 차세대 AI 모델 '제미니 3.0' 전격 공개… '인간 수준의 추론 능력' 목표
날짜: 2025-09-11
내용: 구글이 현지 시간 10일, 자사의 연례 개발자 컨퍼런스에서 차세대 대규모 언어 모델(LLM)인 '제미니 3.0'을 공개했습니다. 이번 모델은 텍스트, 이미지, 오디오를 동시에 이해...


기사 2:
제목: 국내 스타트업 'AURA팀', AI 기반 URL 분석 서비스로 CES 2026 혁신상 수상
날짜: 2025-09-11
내용: 대한민국의 AI 스타트업 'AURA팀'이 개발한 동명의 서비스 'AURA'가 세계 최대 IT·가전 전시회 'CES 2026'에서 소프트웨어 및 모바일 앱 부문 혁신상을 수상하는 쾌...


기사 3:
제목: 비동기 처리 기술, 대규모 AI 서비스의 필수 요건으로 자리매김
날짜: 2025-09-10
내용: ChatGPT와 같은 대규모 AI 서비스가 대중화되면서, 수많은 사용자 요청을 효율적으로 처리하기 위한 백엔드 기술의 중요성이 커지고 있습니다. 특히 사용자의 요청을 기다리게 하지...




## 📁 Output 폴더 구조 생성


In [4]:
def create_output_structure(category, date_str=None):
    """
    카테고리별 output 폴더 구조를 생성하는 함수
    
    Args:
        category (str): 카테고리명 (예: IT/기술)
        date_str (str): 날짜 문자열 (기본값: 오늘 날짜)
    
    Returns:
        dict: 생성된 폴더 경로들
    """
    if date_str is None:
        date_str = datetime.now().strftime("%Y%m%d")
    
    # 카테고리명을 파일명에 적합하게 변환
    safe_category = category.replace("/", "_").replace(" ", "_")
    
    # 기본 경로 설정 (data/output으로 변경)
    base_path = "data/output"
    category_path = os.path.join(base_path, f"{safe_category}_{date_str}")
    
    # 하위 폴더들 생성
    folders = {
        "base": base_path,
        "category": category_path,
        "reports": os.path.join(category_path, "reports"),
        "scripts": os.path.join(category_path, "scripts"),
        "audio": os.path.join(category_path, "audio"),
        "processed": os.path.join(category_path, "processed")
    }
    
    # 폴더들 생성
    for folder_name, folder_path in folders.items():
        os.makedirs(folder_path, exist_ok=True)
        print(f"📁 {folder_name} 폴더 생성: {folder_path}")
    
    return folders

# 폴더 구조 생성 테스트
folders = create_output_structure("IT/기술")
print(f"\n✅ 폴더 구조 생성 완료!")
print(f"📂 카테고리 폴더: {folders['category']}")


📁 base 폴더 생성: data/output
📁 category 폴더 생성: data/output\IT_기술_20250914
📁 reports 폴더 생성: data/output\IT_기술_20250914\reports
📁 scripts 폴더 생성: data/output\IT_기술_20250914\scripts
📁 audio 폴더 생성: data/output\IT_기술_20250914\audio
📁 processed 폴더 생성: data/output\IT_기술_20250914\processed

✅ 폴더 구조 생성 완료!
📂 카테고리 폴더: data/output\IT_기술_20250914


## 📊 종합 보고서 생성 함수


In [5]:
## 📝 보고서 생성 템플릿 정의

# 종합 보고서 템플릿
COMPREHENSIVE_REPORT_TEMPLATE = """# 📊 {category} 종합 보고서

## 📋 요약
{summary}

## 📰 주요 뉴스 내용
{detailed_content}

## 🔗 관련 뉴스 링크
{news_links}

---
**보고서 생성일**: {report_date}
**분석 기사 수**: {total_articles}개
"""

# 요약 템플릿
SUMMARY_TEMPLATE = """{category} 분야에서 총 {total_articles}개의 주요 뉴스가 발표되었습니다. 주요 키워드는 {key_points} 등입니다."""

# 상세 내용 템플릿
DETAILED_CONTENT_TEMPLATE = """
### {title}
**발표일**: {date}
{content}

---
"""


## 🎙️ 2인 대화형 팟캐스트 대본 생성 함수


In [6]:
def generate_comprehensive_report(file_path):
    """
    JSON 파일을 읽어서 모든 뉴스를 하나의 종합 보고서로 생성하는 함수
    
    Args:
        file_path (str): JSON 파일 경로
    
    Returns:
        str: 생성된 종합 보고서 텍스트
    """
    
    # JSON 파일 읽기
    with open(file_path, 'r', encoding='utf-8') as file:
        data = json.load(file)
    
    # PromptTemplate 객체들 생성
    comprehensive_prompt = PromptTemplate(
        input_variables=["category", "summary", "detailed_content", "news_links", "report_date", "total_articles"],
        template=COMPREHENSIVE_REPORT_TEMPLATE
    )
    
    summary_prompt = PromptTemplate(
        input_variables=["category", "total_articles", "key_points"],
        template=SUMMARY_TEMPLATE
    )
    
    detailed_prompt = PromptTemplate(
        input_variables=["title", "date", "content"],
        template=DETAILED_CONTENT_TEMPLATE
    )
    
    # 키 포인트 추출 (제목들로부터)
    key_points = ", ".join([article['title'] for article in data['articles'][:2]])  # 처음 2개 기사 제목
    if len(data['articles']) > 2:
        key_points += " 등"
    
    # 요약 생성
    summary = summary_prompt.format(
        category=data['category'],
        total_articles=len(data['articles']),
        key_points=key_points
    )
    
    # 상세 내용 생성 (모든 기사 내용을 하나로)
    detailed_content = ""
    for article in data['articles']:
        detailed_content += detailed_prompt.format(
            title=article['title'],
            date=article['date'],
            content=article['text']
        )
    
    # 뉴스 링크 생성
    news_links = ""
    for i, article in enumerate(data['articles'], 1):
        news_links += f"{i}. [{article['title'][:50]}...]({article['news_url']})\\n"
    
    # 최종 종합 보고서 생성
    final_report = comprehensive_prompt.format(
        category=data['category'],
        summary=summary,
        detailed_content=detailed_content,
        news_links=news_links,
        report_date=datetime.now().strftime("%Y-%m-%d"),
        total_articles=len(data['articles'])
    )
    
    return final_report


In [7]:
def generate_podcast_script_from_md(md_file_path, host1_name="김테크", host2_name="박AI"):
    """
    MD 보고서 파일을 읽어서 2인 대화형 팟캐스트 대본을 생성하는 함수
    
    Args:
        md_file_path (str): MD 보고서 파일 경로
        host1_name (str): 첫 번째 호스트 이름
        host2_name (str): 두 번째 호스트 이름
    
    Returns:
        str: 생성된 2인 대화형 팟캐스트 대본
    """
    
    # MD 파일 읽기
    with open(md_file_path, 'r', encoding='utf-8') as file:
        md_content = file.read()
    
    # 전체 팟캐스트 대본 생성 프롬프트
    full_script_prompt = PromptTemplate(
        input_variables=["md_content", "host1_name", "host2_name"],
        template="""
다음 뉴스 보고서를 바탕으로 2인 대화형 팟캐스트 대본을 생성해주세요.

**보고서 내용:**
{md_content}

**요구사항:**
1. 진행자: {host1_name}, {host2_name}
2. 자연스러운 대화 형식
3. 각 뉴스에 대한 흥미로운 의견과 분석 포함
4. 약 15분 분량
5. 인트로, 메인 콘텐츠, 심화 분석, 마무리 구조

**대본 형식:**
{host1_name}: (대사)
{host2_name}: (대사)

자연스럽고 흥미로운 팟캐스트 대본을 작성해주세요.
"""
    )
    
    # LLM 체인 생성
    chain = full_script_prompt | llm | StrOutputParser()
    
    try:
        # LLM으로 대본 생성
        script_content = chain.invoke({
            "md_content": md_content,
            "host1_name": host1_name,
            "host2_name": host2_name
        })
        
        # 에피소드 정보 추가
        episode_info = f"""# 🎙️ AI 생성 팟캐스트 대본 (2인 대화형)

## 📻 에피소드 정보
- **제목**: IT/기술 분야 최신 동향 - {datetime.now().strftime('%m월 %d일')} 뉴스
- **날짜**: {datetime.now().strftime('%Y년 %m월 %d일')}
- **주제**: IT/기술 분야 최신 동향
- **진행자**: {host1_name}, {host2_name}
- **소요 시간**: 약 15분
- **생성 방식**: AI 모델 (Gemini-1.5-flash)

---

{script_content}

---

## 📝 방송 노트
**생성 정보:**
- 생성일: {datetime.now().strftime("%Y-%m-%d %H:%M:%S")}
- AI 모델: Gemini-1.5-flash
- 진행자: {host1_name}, {host2_name}
- 원본 보고서: {md_file_path}
"""
        
        return episode_info
        
    except Exception as e:
        print(f"❌ LLM 호출 중 오류 발생: {e}")
        print("⚠️ API 키를 확인하거나 네트워크 연결을 확인해주세요.")
        return None


## 🚀 통합 워크플로우 실행


In [8]:
def save_podcast_script_to_output(script_content, category, host1_name, host2_name, filename=None):
    """팟캐스트 대본을 output 폴더에 저장하는 함수"""
    folders = create_output_structure(category)
    if filename is None:
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        safe_category = category.replace("/", "_").replace(" ", "_")
        filename = f"{safe_category}_2인대화형_팟캐스트_{timestamp}.md"
    file_path = os.path.join(folders['scripts'], filename)
    with open(file_path, 'w', encoding='utf-8') as f:
        f.write(script_content)
    print(f"🎙️ 팟캐스트 대본이 저장되었습니다: {file_path}")
    return file_path

# 업데이트된 통합 워크플로우 실행
def run_complete_workflow(json_file_path, category, host1_name="김테크", host2_name="박AI"):
    """JSON 파일부터 팟캐스트 대본까지 전체 워크플로우를 실행하는 함수"""
    print("🚀 통합 워크플로우 시작!")
    print(f"📂 카테고리: {category}")
    print(f"🎙️ 진행자: {host1_name}, {host2_name}")
    print("-" * 50)
    
    # 1단계: 보고서 생성
    print("📊 1단계: 종합 보고서 생성 중...")
    report_content = generate_comprehensive_report(json_file_path)
    report_file_path = save_report_to_output(report_content, category)
    
    # 2단계: 팟캐스트 대본 생성
    print("\n🎙️ 2단계: 2인 대화형 팟캐스트 대본 생성 중...")
    script_content = generate_podcast_script_from_md(report_file_path, host1_name, host2_name)
    script_file_path = save_podcast_script_to_output(script_content, category, host1_name, host2_name)
    
    print("\n✅ 워크플로우 완료!")
    print(f"📄 보고서: {report_file_path}")
    print(f"🎙️ 대본: {script_file_path}")
    
    return {
        "category": category,
        "hosts": [host1_name, host2_name],
        "report_file": report_file_path,
        "script_file": script_file_path,
        "folders": create_output_structure(category)
    }

# 수정된 팟캐스트 대본 생성 테스트
print("🔧 수정된 팟캐스트 대본 생성 테스트")
print("-" * 50)

# 기존 보고서 파일 경로
existing_report = "data/output/IT_기술_20250914/reports/IT_기술_종합보고서_20250914_200813.md"

# 수정된 팟캐스트 대본 생성
script_content = generate_podcast_script_from_md(existing_report, "김테크", "박AI")
script_file_path = save_podcast_script_to_output(script_content, "IT/기술", "김테크", "박AI")

print(f"\n✅ 수정된 팟캐스트 대본 생성 완료!")
print(f"🎙️ 대본: {script_file_path}")

# 통합 워크플로우 실행 (선택사항)
# workflow_result = run_complete_workflow('data/input/test.json', "IT/기술", "김테크", "박AI")

print("\n" + "="*60)
print("🎯 생성된 파일 구조:")
print(f"📂 data/output/IT_기술_20250914")
print(f"  📁 reports/")
print(f"    📄 IT_기술_종합보고서_20250914_200813.md")
print(f"  📁 scripts/")
print(f"    📄 {os.path.basename(script_file_path)}")
print(f"  📁 audio/ (향후 음성 파일용)")
print(f"  📁 processed/ (향후 전처리 파일용)")
print("="*60)


🔧 수정된 팟캐스트 대본 생성 테스트
--------------------------------------------------
📁 base 폴더 생성: data/output
📁 category 폴더 생성: data/output\IT_기술_20250914
📁 reports 폴더 생성: data/output\IT_기술_20250914\reports
📁 scripts 폴더 생성: data/output\IT_기술_20250914\scripts
📁 audio 폴더 생성: data/output\IT_기술_20250914\audio
📁 processed 폴더 생성: data/output\IT_기술_20250914\processed
🎙️ 팟캐스트 대본이 저장되었습니다: data/output\IT_기술_20250914\scripts\IT_기술_2인대화형_팟캐스트_20250914_232711.md

✅ 수정된 팟캐스트 대본 생성 완료!
🎙️ 대본: data/output\IT_기술_20250914\scripts\IT_기술_2인대화형_팟캐스트_20250914_232711.md

🎯 생성된 파일 구조:
📂 data/output/IT_기술_20250914
  📁 reports/
    📄 IT_기술_종합보고서_20250914_200813.md
  📁 scripts/
    📄 IT_기술_2인대화형_팟캐스트_20250914_232711.md
  📁 audio/ (향후 음성 파일용)
  📁 processed/ (향후 전처리 파일용)


In [9]:
def save_report_to_output(report_content, category, filename=None):
    """보고서를 output 폴더에 저장하는 함수"""
    folders = create_output_structure(category)
    if filename is None:
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        safe_category = category.replace("/", "_").replace(" ", "_")
        filename = f"{safe_category}_종합보고서_{timestamp}.md"
    file_path = os.path.join(folders['reports'], filename)
    with open(file_path, 'w', encoding='utf-8') as f:
        f.write(report_content)
    print(f"📄 보고서가 저장되었습니다: {file_path}")
    return file_path

def run_complete_workflow(json_file_path, category, host1_name="김테크", host2_name="박AI"):
    """JSON 파일부터 팟캐스트 대본까지 전체 워크플로우를 실행하는 함수"""
    print("🚀 통합 워크플로우 시작!")
    print(f"📂 카테고리: {category}")
    print(f"🎙️ 진행자: {host1_name}, {host2_name}")
    print("-" * 50)
    
    # 1단계: 보고서 생성
    print("📊 1단계: 종합 보고서 생성 중...")
    report_content = generate_comprehensive_report(json_file_path)
    report_file_path = save_report_to_output(report_content, category)
    
    print("\n✅ 워크플로우 완료!")
    print(f"📄 보고서: {report_file_path}")
    
    return {
        "category": category,
        "hosts": [host1_name, host2_name],
        "report_file": report_file_path,
        "folders": create_output_structure(category)
    }

# 통합 워크플로우 실행
workflow_result = run_complete_workflow('data/input/test.json', "IT/기술", "김테크", "박AI")

print("\n" + "="*60)
print("🎯 생성된 파일 구조:")
print(f"📂 {workflow_result['folders']['category']}")
print(f"  📁 reports/")
print(f"    📄 {os.path.basename(workflow_result['report_file'])}")
print(f"  📁 scripts/ (팟캐스트 대본용)")
print(f"  📁 audio/ (향후 음성 파일용)")
print(f"  📁 processed/ (향후 전처리 파일용)")
print("="*60)


🚀 통합 워크플로우 시작!
📂 카테고리: IT/기술
🎙️ 진행자: 김테크, 박AI
--------------------------------------------------
📊 1단계: 종합 보고서 생성 중...
📁 base 폴더 생성: data/output
📁 category 폴더 생성: data/output\IT_기술_20250914
📁 reports 폴더 생성: data/output\IT_기술_20250914\reports
📁 scripts 폴더 생성: data/output\IT_기술_20250914\scripts
📁 audio 폴더 생성: data/output\IT_기술_20250914\audio
📁 processed 폴더 생성: data/output\IT_기술_20250914\processed
📄 보고서가 저장되었습니다: data/output\IT_기술_20250914\reports\IT_기술_종합보고서_20250914_232711.md

✅ 워크플로우 완료!
📄 보고서: data/output\IT_기술_20250914\reports\IT_기술_종합보고서_20250914_232711.md
📁 base 폴더 생성: data/output
📁 category 폴더 생성: data/output\IT_기술_20250914
📁 reports 폴더 생성: data/output\IT_기술_20250914\reports
📁 scripts 폴더 생성: data/output\IT_기술_20250914\scripts
📁 audio 폴더 생성: data/output\IT_기술_20250914\audio
📁 processed 폴더 생성: data/output\IT_기술_20250914\processed

🎯 생성된 파일 구조:
📂 data/output\IT_기술_20250914
  📁 reports/
    📄 IT_기술_종합보고서_20250914_232711.md
  📁 scripts/ (팟캐스트 대본용)
  📁 audio/ (향후 음성 파일용)
  📁 processed/ 

In [10]:
## 🤖 실제 LLM을 사용한 팟캐스트 대본 생성

def generate_podcast_script_with_llm(md_file_path, host1_name="김테크", host2_name="박AI"):
    """
    실제 LLM을 사용하여 2인 대화형 팟캐스트 대본을 생성하는 함수
    
    Args:
        md_file_path (str): MD 보고서 파일 경로
        host1_name (str): 첫 번째 호스트 이름
        host2_name (str): 두 번째 호스트 이름
    
    Returns:
        str: 생성된 2인 대화형 팟캐스트 대본
    """
    
    # MD 파일 읽기
    with open(md_file_path, 'r', encoding='utf-8') as file:
        md_content = file.read()
    
    # 전체 팟캐스트 대본 생성 프롬프트
    full_script_prompt = PromptTemplate(
        input_variables=["md_content", "host1_name", "host2_name"],
        template="""
다음 뉴스 보고서를 바탕으로 2인 대화형 팟캐스트 대본을 생성해주세요.

**보고서 내용:**
{md_content}

**요구사항:**
1. 진행자: {host1_name}, {host2_name}
2. 자연스러운 대화 형식
3. 각 뉴스에 대한 흥미로운 의견과 분석 포함
4. 약 15분 분량
5. 인트로, 메인 콘텐츠, 심화 분석, 마무리 구조

**대본 형식:**
{host1_name}: (대사)
{host2_name}: (대사)

자연스럽고 흥미로운 팟캐스트 대본을 작성해주세요.
"""
    )
    
    # LLM 체인 생성
    chain = full_script_prompt | llm | StrOutputParser()
    
    try:
        # LLM으로 대본 생성
        script_content = chain.invoke({
            "md_content": md_content,
            "host1_name": host1_name,
            "host2_name": host2_name
        })
        
        # 에피소드 정보 추가
        episode_info = f"""# 🎙️ AI 생성 팟캐스트 대본 (2인 대화형)

## 📻 에피소드 정보
- **제목**: IT/기술 분야 최신 동향 - {datetime.now().strftime('%m월 %d일')} 뉴스
- **날짜**: {datetime.now().strftime('%Y년 %m월 %d일')}
- **주제**: IT/기술 분야 최신 동향
- **진행자**: {host1_name}, {host2_name}
- **소요 시간**: 약 15분
- **생성 방식**: AI 모델 (GPT-4o-mini)

---

{script_content}

---

## 📝 방송 노트
**생성 정보:**
- 생성일: {datetime.now().strftime("%Y-%m-%d %H:%M:%S")}
- AI 모델: GPT-4o-mini
- 진행자: {host1_name}, {host2_name}
- 원본 보고서: {md_file_path}
"""
        
        return episode_info
        
    except Exception as e:
        print(f"❌ LLM 호출 중 오류 발생: {e}")
        print("⚠️ API 키를 확인하거나 네트워크 연결을 확인해주세요.")
        return None

# 테스트 실행 (API 키가 설정된 경우에만)
print("🤖 LLM 기반 팟캐스트 대본 생성 테스트")
print("-" * 50)

# 기존 보고서 파일 경로
existing_report = "data/output/IT_기술_20250914/reports/IT_기술_종합보고서_20250914_200813.md"

# LLM 기반 대본 생성 테스트
script_content = generate_podcast_script_with_llm(existing_report, "김테크", "박AI")
if script_content:
    # 대본 저장
    folders = create_output_structure("IT/기술")
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    script_file_path = os.path.join(folders['scripts'], f"IT_기술_LLM_팟캐스트_{timestamp}.md")
    
    with open(script_file_path, 'w', encoding='utf-8') as f:
        f.write(script_content)
    
    print(f"✅ LLM 기반 팟캐스트 대본이 생성되었습니다: {script_file_path}")
else:
    print("❌ 대본 생성에 실패했습니다.")

🤖 LLM 기반 팟캐스트 대본 생성 테스트
--------------------------------------------------
📁 base 폴더 생성: data/output
📁 category 폴더 생성: data/output\IT_기술_20250914
📁 reports 폴더 생성: data/output\IT_기술_20250914\reports
📁 scripts 폴더 생성: data/output\IT_기술_20250914\scripts
📁 audio 폴더 생성: data/output\IT_기술_20250914\audio
📁 processed 폴더 생성: data/output\IT_기술_20250914\processed
✅ LLM 기반 팟캐스트 대본이 생성되었습니다: data/output\IT_기술_20250914\scripts\IT_기술_LLM_팟캐스트_20250914_232721.md


In [11]:
## 📊 LLM 기반 종합 보고서 생성

def generate_comprehensive_report_with_llm(json_file_path):
    """
    JSON 파일을 읽어서 LLM을 사용하여 종합 보고서를 생성하는 함수
    
    Args:
        json_file_path (str): JSON 파일 경로
    
    Returns:
        str: 생성된 종합 보고서 텍스트
    """
    
    # JSON 파일 읽기
    with open(json_file_path, 'r', encoding='utf-8') as file:
        data = json.load(file)
    
    # 보고서 생성 프롬프트
    report_prompt = PromptTemplate(
        input_variables=["category", "articles_data"],
        template="""
다음 뉴스 기사들을 바탕으로 전문적이고 체계적인 종합 보고서를 작성해주세요.

**카테고리**: {category}

**뉴스 기사들**:
{articles_data}

**보고서 요구사항**:
1. 마크다운 형식으로 작성
2. 요약, 주요 내용, 분석 섹션 포함
3. 각 뉴스의 핵심 내용과 시사점 정리
4. 전문적이면서도 이해하기 쉬운 문체
5. 구조화된 형태로 정리

다음 형식을 따라 작성해주세요:

# 📊 [카테고리] 종합 보고서

## 📋 요약
[전체적인 요약]

## 📰 주요 뉴스 내용
### [뉴스 제목 1]
**발표일**: [날짜]
[내용 요약]

### [뉴스 제목 2]
**발표일**: [날짜]
[내용 요약]

## 🔍 종합 분석
[전체적인 분석과 시사점]

---
**보고서 생성일**: {report_date}
**분석 기사 수**: {total_articles}개
"""
    )
    
    # 기사 데이터를 문자열로 변환
    articles_text = ""
    for i, article in enumerate(data['articles'], 1):
        articles_text += f"""
{i}. 제목: {article['title']}
   날짜: {article['date']}
   내용: {article['text'][:500]}...
   URL: {article['news_url']}

"""
    
    # LLM 체인 생성
    chain = report_prompt | llm | StrOutputParser()
    
    try:
        # LLM으로 보고서 생성
        report_content = chain.invoke({
            "category": data['category'],
            "articles_data": articles_text,
            "report_date": datetime.now().strftime("%Y-%m-%d"),
            "total_articles": len(data['articles'])
        })
        
        return report_content
        
    except Exception as e:
        print(f"❌ LLM 호출 중 오류 발생: {e}")
        print("⚠️ API 키를 확인하거나 네트워크 연결을 확인해주세요.")
        return None

# 테스트 실행
print("📊 LLM 기반 종합 보고서 생성 테스트")
print("-" * 50)

if llm is not None:
    report_content = generate_comprehensive_report_with_llm('data/input/test.json')
    if report_content:
        # 보고서 저장
        folders = create_output_structure("IT/기술")
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        report_file_path = os.path.join(folders['reports'], f"IT_기술_LLM_종합보고서_{timestamp}.md")
        
        with open(report_file_path, 'w', encoding='utf-8') as f:
            f.write(report_content)
        
        print(f"✅ LLM 기반 종합 보고서가 생성되었습니다: {report_file_path}")
    else:
        print("❌ 보고서 생성에 실패했습니다.")
else:
    print("⚠️ Google API 키를 설정해주세요.")


📊 LLM 기반 종합 보고서 생성 테스트
--------------------------------------------------
📁 base 폴더 생성: data/output
📁 category 폴더 생성: data/output\IT_기술_20250914
📁 reports 폴더 생성: data/output\IT_기술_20250914\reports
📁 scripts 폴더 생성: data/output\IT_기술_20250914\scripts
📁 audio 폴더 생성: data/output\IT_기술_20250914\audio
📁 processed 폴더 생성: data/output\IT_기술_20250914\processed
✅ LLM 기반 종합 보고서가 생성되었습니다: data/output\IT_기술_20250914\reports\IT_기술_LLM_종합보고서_20250914_232730.md


In [12]:
## 🎵 TTS (Text-to-Speech) 음성 생성 시스템

def text_to_speech(text, voice_name="ko-KR-Standard-A", output_file=None):
    """
    텍스트를 음성으로 변환하는 함수
    
    Args:
        text (str): 변환할 텍스트
        voice_name (str): 사용할 음성 (기본값: 한국어 남성)
        output_file (str): 출력 파일 경로 (기본값: 자동 생성)
    
    Returns:
        str: 생성된 음성 파일 경로
    """
    
    if tts_client is None:
        print("❌ TTS 클라이언트가 초기화되지 않았습니다.")
        return None
    
    try:
        # 음성 설정
        synthesis_input = texttospeech.SynthesisInput(text=text)
        
        # 음성 선택 (한국어)
        voice = texttospeech.VoiceSelectionParams(
            language_code="ko-KR",
            name=voice_name,
            ssml_gender=texttospeech.SsmlVoiceGender.MALE if "A" in voice_name else texttospeech.SsmlVoiceGender.FEMALE
        )
        
        # 오디오 설정
        audio_config = texttospeech.AudioConfig(
            audio_encoding=texttospeech.AudioEncoding.MP3,
            speaking_rate=1.0,  # 말하기 속도
            pitch=0.0,  # 음높이
            volume_gain_db=0.0  # 볼륨
        )
        
        # TTS 요청
        response = tts_client.synthesize_speech(
            input=synthesis_input,
            voice=voice,
            audio_config=audio_config
        )
        
        # 파일 저장
        if output_file is None:
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            output_file = f"temp_audio_{timestamp}.mp3"
        
        with open(output_file, "wb") as out:
            out.write(response.audio_content)
        
        print(f"🎵 음성 파일 생성 완료: {output_file}")
        return output_file
        
    except Exception as e:
        print(f"❌ TTS 변환 중 오류 발생: {e}")
        return None

def parse_podcast_script(script_content):
    """
    팟캐스트 대본을 파싱하여 대화별로 분리하는 함수
    
    Args:
        script_content (str): 팟캐스트 대본 내용
    
    Returns:
        list: 대화 리스트 [{"speaker": "김테크", "text": "대사 내용"}, ...]
    """
    
    # 대본에서 실제 대화 부분만 추출 (인트로, 메인 콘텐츠, 심화 분석, 마무리)
    lines = script_content.split('\n')
    dialogues = []
    
    # 대화 패턴 찾기 (진행자: 대사)
    dialogue_pattern = r'^(김테크|박AI):\s*(.+)$'
    
    for line in lines:
        line = line.strip()
        if line and not line.startswith('#') and not line.startswith('**') and not line.startswith('---'):
            match = re.match(dialogue_pattern, line)
            if match:
                speaker = match.group(1)
                text = match.group(2).strip()
                if text:
                    dialogues.append({
                        "speaker": speaker,
                        "text": text
                    })
    
    print(f"📝 총 {len(dialogues)}개의 대화를 찾았습니다.")
    return dialogues

def generate_podcast_audio(script_file_path, output_dir=None):
    """
    팟캐스트 대본을 음성 파일로 변환하는 함수
    
    Args:
        script_file_path (str): 대본 파일 경로
        output_dir (str): 출력 디렉토리 (기본값: 자동 생성)
    
    Returns:
        str: 최종 음성 파일 경로
    """
    
    if tts_client is None:
        print("❌ TTS 클라이언트가 초기화되지 않았습니다.")
        return None
    
    try:
        # 대본 파일 읽기
        with open(script_file_path, 'r', encoding='utf-8') as f:
            script_content = f.read()
        
        # 대본 파싱
        dialogues = parse_podcast_script(script_content)
        
        if not dialogues:
            print("❌ 대화를 찾을 수 없습니다.")
            return None
        
        # 출력 디렉토리 설정
        if output_dir is None:
            output_dir = "data/output/IT_기술_20250914/audio"
            os.makedirs(output_dir, exist_ok=True)
        
        # 음성 파일들 저장할 리스트
        audio_files = []
        
        # 각 대화를 음성으로 변환
        for i, dialogue in enumerate(dialogues):
            speaker = dialogue["speaker"]
            text = dialogue["text"]
            
            # 진행자별 음성 선택
            if speaker == "김테크":
                voice_name = "ko-KR-Standard-A"  # 남성 음성
            else:  # 박AI
                voice_name = "ko-KR-Standard-C"  # 여성 음성
            
            # 임시 파일명
            temp_file = os.path.join(output_dir, f"temp_{i:03d}_{speaker}.mp3")
            
            print(f"🎤 {speaker}: {text[:50]}...")
            
            # TTS 변환
            audio_file = text_to_speech(text, voice_name, temp_file)
            if audio_file:
                audio_files.append(audio_file)
        
        if not audio_files:
            print("❌ 음성 파일 생성에 실패했습니다.")
            return None
        
        # 음성 파일들 병합
        print("🔗 음성 파일들을 병합 중...")
        final_audio = merge_audio_files(audio_files, output_dir)
        
        # 임시 파일들 삭제
        for temp_file in audio_files:
            if os.path.exists(temp_file):
                os.remove(temp_file)
        
        print(f"✅ 팟캐스트 음성 파일 생성 완료: {final_audio}")
        return final_audio
        
    except Exception as e:
        print(f"❌ 팟캐스트 음성 생성 중 오류 발생: {e}")
        return None

def merge_audio_files(audio_files, output_dir):
    """
    여러 음성 파일을 하나로 병합하는 함수
    
    Args:
        audio_files (list): 병합할 음성 파일 경로들
        output_dir (str): 출력 디렉토리
    
    Returns:
        str: 병합된 음성 파일 경로
    """
    
    try:
        # 첫 번째 파일을 기준으로 시작
        combined = AudioSegment.from_mp3(audio_files[0])
        
        # 나머지 파일들을 순차적으로 병합
        for audio_file in audio_files[1:]:
            audio = AudioSegment.from_mp3(audio_file)
            # 0.5초 간격 추가
            combined += AudioSegment.silent(duration=500) + audio
        
        # 음성 정규화
        combined = normalize(combined)
        
        # 최종 파일 저장
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        output_file = os.path.join(output_dir, f"IT_기술_팟캐스트_{timestamp}.mp3")
        
        combined.export(output_file, format="mp3")
        
        print(f"🎵 음성 파일 병합 완료: {output_file}")
        return output_file
        
    except Exception as e:
        print(f"❌ 음성 파일 병합 중 오류 발생: {e}")
        return None


In [ ]:
# ## 🎙️ TTS 통합 팟캐스트 생성 테스트

# # 기존에 생성된 팟캐스트 대본으로 음성 파일 생성 테스트
# print("🎵 TTS 팟캐스트 음성 생성 테스트")
# print("-" * 50)

# # 기존 대본 파일 경로
# script_file = "data/output/IT_기술_20250914/scripts/IT_기술_LLM_팟캐스트_20250914_205135.md"

# # TTS로 음성 파일 생성
# if tts_client is not None:
#     audio_file = generate_podcast_audio(script_file)
    
#     if audio_file:
#         print(f"\n✅ 팟캐스트 음성 파일이 성공적으로 생성되었습니다!")
#         print(f"🎵 음성 파일: {audio_file}")
        
#         # 파일 크기 확인
#         file_size = os.path.getsize(audio_file) / (1024 * 1024)  # MB
#         print(f"📊 파일 크기: {file_size:.2f} MB")
        
#     else:
#         print("❌ 음성 파일 생성에 실패했습니다.")
# else:
#     print("⚠️ Google TTS 클라이언트를 먼저 설정해주세요.")
#     print("📝 Google Cloud 인증 및 라이브러리 설치가 필요합니다.")


🎵 TTS 팟캐스트 음성 생성 테스트
--------------------------------------------------
📝 총 18개의 대화를 찾았습니다.
🎤 김테크: 안녕하세요, 여러분! Tech Talk의 김테크입니다....
🎵 음성 파일 생성 완료: data/output/IT_기술_20250914/audio\temp_000_김테크.mp3
🎤 박AI: 그리고 AI 전문가 박AI입니다!...
🎵 음성 파일 생성 완료: data/output/IT_기술_20250914/audio\temp_001_박AI.mp3
🎤 김테크: 이번 주는 AI 업계에 정말 굵직한 뉴스들이 쏟아졌죠.  구글의 제미니 3.0 공개부터 국...
🎵 음성 파일 생성 완료: data/output/IT_기술_20250914/audio\temp_002_김테크.mp3
🎤 박AI: 맞아요. 특히 비동기 처리 기술의 중요성까지 다시 한번 부각되면서 AI 시장의 미래를 엿볼...
🎵 음성 파일 생성 완료: data/output/IT_기술_20250914/audio\temp_003_박AI.mp3
🎤 김테크: 첫 번째 뉴스는 역시 구글의 제미니 3.0 공개죠.  인간 수준의 추론 능력을 목표로 한다...
🎵 음성 파일 생성 완료: data/output/IT_기술_20250914/audio\temp_004_김테크.mp3
🎤 박AI: 그렇죠.  이미지와 오디오까지 처리하는 멀티모달 기능은  AI의 활용 범위를 엄청나게 넓힐...
🎵 음성 파일 생성 완료: data/output/IT_기술_20250914/audio\temp_005_박AI.mp3
🎤 김테크: 두 번째 뉴스는 국내 스타트업 AURA팀의 CES 2026 혁신상 수상 소식입니다.  AI...
🎵 음성 파일 생성 완료: data/output/IT_기술_20250914/audio\temp_006_김테크.mp3
🎤 박AI: AURA는 단순히 정보를 요약하는 것을 넘어, 오디오 팟캐스트와 실시간 Q&A까지 제공한다...
🎵 음성 파일 생성 완료: da

In [14]:
## 🚀 완전한 TTS 통합 워크플로우

def run_complete_tts_workflow(json_file_path, category, host1_name="김테크", host2_name="박AI"):
    """
    JSON 파일부터 음성 팟캐스트까지 전체 워크플로우를 실행하는 함수
    
    Args:
        json_file_path (str): JSON 파일 경로
        category (str): 카테고리명
        host1_name (str): 첫 번째 호스트 이름
        host2_name (str): 두 번째 호스트 이름
    
    Returns:
        dict: 생성된 파일 경로들과 결과 정보
    """
    
    print("🚀 TTS 통합 워크플로우 시작!")
    print(f"📂 카테고리: {category}")
    print(f"🎙️ 진행자: {host1_name}, {host2_name}")
    print("-" * 50)
    
    # API 키 확인
    if llm is None:
        print("❌ Google API 키를 설정해주세요.")
        return None
    
    if tts_client is None:
        print("❌ Google TTS 클라이언트를 설정해주세요.")
        return None
    
    # 폴더 구조 생성
    folders = create_output_structure(category)
    
    # 1단계: LLM 기반 종합 보고서 생성
    print("📊 1단계: LLM 기반 종합 보고서 생성 중...")
    report_content = generate_comprehensive_report_with_llm(json_file_path)
    
    if not report_content:
        print("❌ 보고서 생성에 실패했습니다.")
        return None
    
    # 보고서 저장
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    safe_category = category.replace("/", "_").replace(" ", "_")
    report_file_path = os.path.join(folders['reports'], f"{safe_category}_LLM_종합보고서_{timestamp}.md")
    
    with open(report_file_path, 'w', encoding='utf-8') as f:
        f.write(report_content)
    
    print(f"✅ 보고서 생성 완료: {report_file_path}")
    
    # 2단계: LLM 기반 팟캐스트 대본 생성
    print("\n🎙️ 2단계: LLM 기반 팟캐스트 대본 생성 중...")
    script_content = generate_podcast_script_with_llm(report_file_path, host1_name, host2_name)
    
    if not script_content:
        print("❌ 대본 생성에 실패했습니다.")
        return None
    
    # 대본 저장
    script_file_path = os.path.join(folders['scripts'], f"{safe_category}_LLM_팟캐스트_{timestamp}.md")
    
    with open(script_file_path, 'w', encoding='utf-8') as f:
        f.write(script_content)
    
    print(f"✅ 대본 생성 완료: {script_file_path}")
    
    # 3단계: TTS 음성 파일 생성
    print("\n🎵 3단계: TTS 음성 파일 생성 중...")
    audio_file = generate_podcast_audio(script_file_path, folders['audio'])
    
    if not audio_file:
        print("❌ 음성 파일 생성에 실패했습니다.")
        return None
    
    print(f"✅ 음성 파일 생성 완료: {audio_file}")
    
    print("\n🎉 TTS 통합 워크플로우 완료!")
    print(f"📄 보고서: {report_file_path}")
    print(f"🎙️ 대본: {script_file_path}")
    print(f"🎵 음성: {audio_file}")
    
    return {
        "category": category,
        "hosts": [host1_name, host2_name],
        "report_file": report_file_path,
        "script_file": script_file_path,
        "audio_file": audio_file,
        "folders": folders,
        "timestamp": timestamp
    }

# TTS 통합 워크플로우 실행
print("🤖 완전한 TTS 통합 팟캐스트 생성 시스템")
print("=" * 60)

if tts_client is not None and llm is not None:
    workflow_result = run_complete_tts_workflow('data/input/test.json', "IT/기술", "김테크", "박AI")
    
    if workflow_result:
        print("\n" + "="*60)
        print("🎯 생성된 파일 구조:")
        print(f"📂 {workflow_result['folders']['category']}")
        print(f"  📁 reports/")
        print(f"    📄 {os.path.basename(workflow_result['report_file'])}")
        print(f"  📁 scripts/")
        print(f"    📄 {os.path.basename(workflow_result['script_file'])}")
        print(f"  📁 audio/")
        print(f"    🎵 {os.path.basename(workflow_result['audio_file'])}")
        print(f"  📁 processed/ (향후 전처리 파일용)")
        print("="*60)
        print("🎊 모든 작업이 성공적으로 완료되었습니다!")
        
        # 파일 크기 정보
        audio_size = os.path.getsize(workflow_result['audio_file']) / (1024 * 1024)
        print(f"\n📊 생성된 파일 정보:")
        print(f"🎵 음성 파일 크기: {audio_size:.2f} MB")
        print(f"⏱️ 예상 재생 시간: 약 15분")
    else:
        print("❌ 워크플로우 실행에 실패했습니다.")
else:
    print("❌ 필요한 API 클라이언트가 설정되지 않았습니다.")
    print("📝 Google API 키와 Google Cloud 인증을 확인해주세요.")


🤖 완전한 TTS 통합 팟캐스트 생성 시스템
🚀 TTS 통합 워크플로우 시작!
📂 카테고리: IT/기술
🎙️ 진행자: 김테크, 박AI
--------------------------------------------------
📁 base 폴더 생성: data/output
📁 category 폴더 생성: data/output\IT_기술_20250914
📁 reports 폴더 생성: data/output\IT_기술_20250914\reports
📁 scripts 폴더 생성: data/output\IT_기술_20250914\scripts
📁 audio 폴더 생성: data/output\IT_기술_20250914\audio
📁 processed 폴더 생성: data/output\IT_기술_20250914\processed
📊 1단계: LLM 기반 종합 보고서 생성 중...
✅ 보고서 생성 완료: data/output\IT_기술_20250914\reports\IT_기술_LLM_종합보고서_20250914_232756.md

🎙️ 2단계: LLM 기반 팟캐스트 대본 생성 중...
✅ 대본 생성 완료: data/output\IT_기술_20250914\scripts\IT_기술_LLM_팟캐스트_20250914_232756.md

🎵 3단계: TTS 음성 파일 생성 중...
📝 총 19개의 대화를 찾았습니다.
🎤 김테크: 안녕하세요, 여러분! 김테크입니다....
🎵 음성 파일 생성 완료: data/output\IT_기술_20250914\audio\temp_000_김테크.mp3
🎤 박AI: 안녕하세요, 박AI입니다. 이번 주도 AI 관련 핫한 뉴스들이 쏟아져 나왔는데요....
🎵 음성 파일 생성 완료: data/output\IT_기술_20250914\audio\temp_001_박AI.mp3
🎤 김테크: 그렇죠. 특히 구글의 새로운 AI 모델 발표부터 국내 스타트업의 쾌거까지, 정말 다채로운 ...
🎵 음성 파일 생성 완료: data/output\IT_기술_20250914\audio

In [15]:
## 🚀 완전한 LLM 기반 통합 워크플로우

def run_complete_llm_workflow(json_file_path, category, host1_name="김테크", host2_name="박AI"):
    """
    JSON 파일부터 팟캐스트 대본까지 전체 워크플로우를 LLM으로 실행하는 함수
    
    Args:
        json_file_path (str): JSON 파일 경로
        category (str): 카테고리명
        host1_name (str): 첫 번째 호스트 이름
        host2_name (str): 두 번째 호스트 이름
    
    Returns:
        dict: 생성된 파일 경로들과 결과 정보
    """
    
    print("🚀 LLM 기반 통합 워크플로우 시작!")
    print(f"📂 카테고리: {category}")
    print(f"🎙️ 진행자: {host1_name}, {host2_name}")
    print("-" * 50)
    
    # API 키 확인
    if llm is None:
        print("❌ Google API 키를 설정해주세요.")
        return None
    
    # 폴더 구조 생성
    folders = create_output_structure(category)
    
    # 1단계: LLM 기반 종합 보고서 생성
    print("📊 1단계: LLM 기반 종합 보고서 생성 중...")
    report_content = generate_comprehensive_report_with_llm(json_file_path)
    
    if not report_content:
        print("❌ 보고서 생성에 실패했습니다.")
        return None
    
    # 보고서 저장
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    safe_category = category.replace("/", "_").replace(" ", "_")
    report_file_path = os.path.join(folders['reports'], f"{safe_category}_LLM_종합보고서_{timestamp}.md")
    
    with open(report_file_path, 'w', encoding='utf-8') as f:
        f.write(report_content)
    
    print(f"✅ 보고서 생성 완료: {report_file_path}")
    
    # 2단계: LLM 기반 팟캐스트 대본 생성
    print("\n🎙️ 2단계: LLM 기반 팟캐스트 대본 생성 중...")
    script_content = generate_podcast_script_with_llm(report_file_path, host1_name, host2_name)
    
    if not script_content:
        print("❌ 대본 생성에 실패했습니다.")
        return None
    
    # 대본 저장
    script_file_path = os.path.join(folders['scripts'], f"{safe_category}_LLM_팟캐스트_{timestamp}.md")
    
    with open(script_file_path, 'w', encoding='utf-8') as f:
        f.write(script_content)
    
    print(f"✅ 대본 생성 완료: {script_file_path}")
    
    print("\n🎉 LLM 기반 워크플로우 완료!")
    print(f"📄 보고서: {report_file_path}")
    print(f"🎙️ 대본: {script_file_path}")
    
    return {
        "category": category,
        "hosts": [host1_name, host2_name],
        "report_file": report_file_path,
        "script_file": script_file_path,
        "folders": folders,
        "timestamp": timestamp
    }

# 통합 워크플로우 실행
print("🤖 완전한 LLM 기반 팟캐스트 생성 시스템")
print("=" * 60)

workflow_result = run_complete_llm_workflow('data/input/test.json', "IT/기술", "김테크", "박AI")

if workflow_result:
    print("\n" + "="*60)
    print("🎯 생성된 파일 구조:")
    print(f"📂 {workflow_result['folders']['category']}")
    print(f"  📁 reports/")
    print(f"    📄 {os.path.basename(workflow_result['report_file'])}")
    print(f"  📁 scripts/")
    print(f"    📄 {os.path.basename(workflow_result['script_file'])}")
    print(f"  📁 audio/ (향후 TTS 음성 파일용)")
    print(f"  📁 processed/ (향후 전처리 파일용)")
    print("="*60)
    print("🎊 모든 작업이 성공적으로 완료되었습니다!")
else:
    print("❌ 워크플로우 실행에 실패했습니다.")
    print("📝 API 키를 확인하고 다시 시도해주세요.")


🤖 완전한 LLM 기반 팟캐스트 생성 시스템
🚀 LLM 기반 통합 워크플로우 시작!
📂 카테고리: IT/기술
🎙️ 진행자: 김테크, 박AI
--------------------------------------------------
📁 base 폴더 생성: data/output
📁 category 폴더 생성: data/output\IT_기술_20250914
📁 reports 폴더 생성: data/output\IT_기술_20250914\reports
📁 scripts 폴더 생성: data/output\IT_기술_20250914\scripts
📁 audio 폴더 생성: data/output\IT_기술_20250914\audio
📁 processed 폴더 생성: data/output\IT_기술_20250914\processed
📊 1단계: LLM 기반 종합 보고서 생성 중...
✅ 보고서 생성 완료: data/output\IT_기술_20250914\reports\IT_기술_LLM_종합보고서_20250914_232827.md

🎙️ 2단계: LLM 기반 팟캐스트 대본 생성 중...
✅ 대본 생성 완료: data/output\IT_기술_20250914\scripts\IT_기술_LLM_팟캐스트_20250914_232827.md

🎉 LLM 기반 워크플로우 완료!
📄 보고서: data/output\IT_기술_20250914\reports\IT_기술_LLM_종합보고서_20250914_232827.md
🎙️ 대본: data/output\IT_기술_20250914\scripts\IT_기술_LLM_팟캐스트_20250914_232827.md

🎯 생성된 파일 구조:
📂 data/output\IT_기술_20250914
  📁 reports/
    📄 IT_기술_LLM_종합보고서_20250914_232827.md
  📁 scripts/
    📄 IT_기술_LLM_팟캐스트_20250914_232827.md
  📁 audio/ (향후 TTS 음성 파일용)
  📁 processed/ (향